# Info

You can run the entire script by press **CTRL+F9**.

For more helpful scripts, check out the quick commands.
*   [Robohood | Quick commands](https://colab.research.google.com/drive1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5?usp=sharing)
*  [Cancel all pending orders](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=ElxYAXz3LWR5&line=1&uniqifier=1)
*  [Close all positions](https://colab.research.google.com/drive/1WsbRD8Rlz_ceSoGQzeGPZX5bNeRLLbz5#scrollTo=LD2LSDnqOlFe&line=1&uniqifier=1)

#Application code
Only needs to be run once per session.

In [ ]:
%pip install robin_stocks

In [ ]:
from robin_stocks import robinhood as r
import itertools
from datetime import datetime
import getpass
import sys

## Funcitons

In [ ]:
def get_spread_cost(front_leg_option, back_leg_option):
  spread_cost = sys.maxsize
  back_ask_price = back_leg_option.get("high_fill_rate_buy_price")
  front_bid_price = front_leg_option.get("high_fill_rate_sell_price")

  if(back_ask_price and front_bid_price):
    spread_cost = round(float(back_ask_price) - float(front_bid_price), 2)

  return spread_cost

In [ ]:
def find_calendar_spreads(options):
    # Create a create new calendar spread for every combination of expiration dates where the short front leg expiration_date is less than the long back leg of the spread
  sorted_options = sorted(options, key=lambda x: (float(x['strike_price']), datetime.strptime(x["expiration_date"],'%Y-%m-%d')))

  options_grouped_by_strike = itertools.groupby(sorted_options, lambda x : x['strike_price'])

  today = datetime.today()

  #  using the front_leg_option and back_leg_option as the front and back legs of the trade, create a debit calendar spread
  #  with a cost of zero
  calendar_spreads = []
  for strike, options in options_grouped_by_strike:
    for (front_leg_option, back_leg_option) in itertools.combinations(options,2):
      front_expiration_date = datetime.strptime(front_leg_option["expiration_date"],'%Y-%m-%d')
      back_expiration_date = datetime.strptime(back_leg_option["expiration_date"],'%Y-%m-%d')
      optionTypeIsSame = front_leg_option["type"] == back_leg_option["type"]

      if (optionTypeIsSame and (front_expiration_date > today) and (front_expiration_date < back_expiration_date)):
        calendar_spreads.append((front_leg_option, back_leg_option, get_spread_cost(front_leg_option, back_leg_option)))

  return calendar_spreads

In [ ]:
def order_calendar_spread(spread, price=0.00, quantity=1, timeInForce='gfd'):
  front_leg_option, back_leg_option, spread_cost = spread
  symbol = front_leg_option["symbol"]
  params = [
            {
            'expirationDate': front_leg_option['expiration_date'],
            'strike': front_leg_option['strike_price'],
            'optionType': front_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'sell',
          },
          {
            'expirationDate': back_leg_option['expiration_date'],
            'strike': back_leg_option['strike_price'],
            'optionType': back_leg_option['type'],
            'quantity': '1',
            'effect': 'open',
            'action': 'buy',
          },
  ]
  
  return r.orders.order_option_spread(direction='debit', price=price, symbol=symbol, quantity=quantity, spread=params, timeInForce=timeInForce)


In [ ]:
def filter_calendar_spreads(spreads, spread_max_cost=.1):

  filtered_spreads = list(filter(lambda spread: spread[2] <= spread_max_cost, spreads))
  filtered_spreads = sorted(filtered_spreads, key=lambda spread: (spread[2]))

  return filtered_spreads

In [ ]:
def get_current_price_of_symbol(symbol="SPY"):
  current_price = round(float(r.stocks.get_latest_price(symbol)[0]))
  print("{} is currently trading at: ${}".format(symbol, current_price))
  return current_price 

In [ ]:
def scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset):
  if(optionType == 'both'):
    optionType = ""

  current_price = get_current_price_of_symbol(symbol)
  min_strike = current_price - min_strike_offset;
  max_strike = current_price + max_strike_offset;

  # Get the list of options available and sort by strike_price and expiration_date
  options = r.options.find_tradable_options(symbol, optionType=optionType, info=None)
  options = list(filter(lambda x: (float(x["strike_price"]) > min_strike) and (float(x["strike_price"]) < max_strike), options))
  return options

In [ ]:
def print_spread(spread):
  (front_leg_option, back_leg_option, spread_cost) = spread
  strike = float(front_leg_option.get("strike_price"))
  print("strike {} | short {} @ {} | long {} @ {} | spread: ${}".format(
      strike,
      front_leg_option.get("type"),
      front_leg_option.get("expiration_date"),
      back_leg_option.get("type"),
      back_leg_option.get("expiration_date"),
      spread_cost))

In [ ]:
def print_spreads(spreads):
  for spread in spreads:
    print_spread(spread)

In [ ]:
def place_calendar_spreads(spreads, price_type, quanity_type, timeInForce):
  price = 0.00
  quantity=1

  if(price_type == "set_for_all"):
    price = float(input("Price for all orders: ") or 0.00)

  if(quanity_type == "set_for_all"):
    quantity = int(input("Quantity for all orders: ") or 0)

  for spread in spreads:
    (front_leg_option, back_leg_option, spread_cost) = spread
    print_spread(spread)

    if(price_type == "set_each"):
      price = float(input("price (limit): ") or 0.00)

    if(price_type == "spread_cost"):
      price = spread_cost

    if(quanity_type == "set_each"):
      quantity = int(input("order size (quantity): ") or 1)
    print("buy {} @ ${}".format(quantity, price))
    results = order_calendar_spread(spread=spread, price=price, quantity=quantity, timeInForce=timeInForce)
    print(results)

In [ ]:
def update_option_market_data(options):
  for item in options:
    marketData = r.options.get_option_market_data_by_id(item['id'])
    if marketData:
        item.update(marketData[0])

# Login

In [ ]:
#@title #Authenticate { vertical-output: true, display-mode: "form" }
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
r.login(username, password)

#Scan for options

> Inputs needed: offset and max offset. By default, the scanner will analize $10 worth of different strikes. 5 above and 5 below.

* **symbol**: example "spy"
* **min strike offset**: how far below the current strike you want to look. The default is 5 dollars.
* **max strike offset**: how far above the current strike you want to look. The 
default is 5 dollars.

In [ ]:
#@title Enter option criteria { run: "auto", vertical-output: true, display-mode: "form" }
# Create a script that places a debit calendar spread for each tradeable option of a given symbol. The cost of the spread needs to be zero.
symbol = "SPY" #@param {type:"string"}
optionType = "both" #@param ["call", "put", "both"]
min_strike_offset = 2 #@param {type:"number"}
max_strike_offset =  2 #@param {type:"number"}

options = scan_for_options(symbol, optionType, min_strike_offset, max_strike_offset)
print("Base on user criteria, you found {} to analize!".format(len(options)))

##Get option market data


> This gets the latest market information for each option found. Data such as ask_price and bid_price are changing all the time. It's a good idea to refresh this often.

In [50]:
update_option_market_data(options)

#Filter calendar spreads

Enter max price you want filter for each spread. This is calculated by subtracting the front leg bid price from the ask price of the back leg. ask_price - bid_price = spread_cost. The default is $0.30.

In [52]:
#@title Filter based on spread cost { run: "auto", vertical-output: true, display-mode: "form" }
max_spread_cost =  0.25 #@param {type:"number"}
calendar_spreads = find_calendar_spreads(options)
filtered_spreads = filter_calendar_spreads(calendar_spreads, max_spread_cost)
print("{} spreads found.".format(len(filtered_spreads)))
print_spreads(filtered_spreads)

10 spreads found.
strike 392.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.23
strike 390.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.24
strike 391.0 | short put @ 2023-01-17 | long put @ 2023-01-18 | spread: $0.24
strike 391.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.24
strike 391.0 | short put @ 2023-01-19 | long put @ 2023-01-20 | spread: $0.24
strike 392.0 | short put @ 2023-01-19 | long put @ 2023-01-20 | spread: $0.24
strike 390.0 | short put @ 2023-01-17 | long put @ 2023-01-18 | spread: $0.25
strike 391.0 | short put @ 2023-01-20 | long put @ 2023-01-23 | spread: $0.25
strike 391.0 | short put @ 2023-01-23 | long put @ 2023-01-24 | spread: $0.25
strike 392.0 | short put @ 2023-01-20 | long put @ 2023-01-23 | spread: $0.25


#Create calendar spread orders

Select the number of order you want to place. Note: the max number of orders you can create in robinhood is 11.

In [53]:
#@title How many orders do you want to place? { run: "auto", vertical-output: true, display-mode: "form" }
max_order_size = 2 #@param {type:"integer"}
top_filtered_spreads = filtered_spreads[:max_order_size] # robinhood only allows 12 orders
print_spreads(top_filtered_spreads)

strike 392.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.23
strike 390.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.24


In [54]:
#@title Place calendar orders based on the following parameters: { vertical-output: true, display-mode: "form" }
price_type = "spread_cost"  # @param ["spread_cost", "set_for_all", "set_each"]
quanity_type = "set_for_all"  # @param ["set_each", "set_for_all"]
timeInForce='gfd'  #@param ["gfd", "gtc"] 
place_calendar_spreads(top_filtered_spreads, price_type, quanity_type, timeInForce)

strike 392.0 | short put @ 2023-01-18 | long put @ 2023-01-19 | spread: $0.23
buy 1 @ $0.23
{'account_number': '5UI31739', 'cancel_url': 'https://api.robinhood.com/options/orders/63be79b7-70ca-4141-90f1-a8786ebeeb22/cancel/', 'canceled_quantity': '0.00000', 'created_at': '2023-01-11T08:56:23.560431Z', 'direction': 'debit', 'id': '63be79b7-70ca-4141-90f1-a8786ebeeb22', 'legs': [{'executions': [], 'id': '63be79b7-7e84-4cf1-a03d-1e410a5266c8', 'option': 'https://api.robinhood.com/options/instruments/51b1b461-8b6b-4255-8397-55800975d3c5/', 'position_effect': 'open', 'ratio_quantity': 1, 'side': 'buy', 'expiration_date': '2023-01-19', 'strike_price': '392.0000', 'option_type': 'put', 'long_strategy_code': '51b1b461-8b6b-4255-8397-55800975d3c5_L1', 'short_strategy_code': '51b1b461-8b6b-4255-8397-55800975d3c5_S1'}, {'executions': [], 'id': '63be79b7-660d-445b-9dab-df7a208317fe', 'option': 'https://api.robinhood.com/options/instruments/76b397ef-e374-4605-80e6-bc794fa3e929/', 'position_effect':